### CSV & EXCEL FILES -Structured Data

In [2]:
import pandas as pd
import os



In [ ]:

# Save DataFrame to CSV

os.makedirs("data/structured_files", exist_ok=True)
# create sample data
df = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'City': ['New York', 'Los Angeles', 'Chicago']
})
# Save DataFrame to CSV
df.to_csv('data/structured_files/sample_data.csv', index=False)

In [ ]:
# Save DataFrame to excel

#create sample data
DataFrame = {
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'City': ['New York', 'Los Angeles', 'Chicago']
}

with pd.ExcelWriter('data/structured_files/sample_data.xlsx') as excel_writer:
    # create a DataFrame from the sample data
    DataFrame = pd.DataFrame(DataFrame)
    # Write DataFrame to Excel
    DataFrame.to_excel(excel_writer, sheet_name='Sheet1', index=False)
   # pd.DataFrame(DataFrame).DataFrame.to_excel(excel_writer, sheet_name='Sheet1', index=False)


In [10]:
# Save as Excel file with multiple sheets
with pd.ExcelWriter('data/structured_files/sample_data_multiple_sheets.xlsx') as excel_writer:
    # create a DataFrame from the sample data
    DataFrame = pd.DataFrame(DataFrame)
    # Write DataFrame to Excel
    DataFrame.to_excel(excel_writer, sheet_name='Sheet1', index=False)
    DataFrame.to_excel(excel_writer, sheet_name='Sheet2', index=False)

### CSV PROCESSING

In [ ]:
from langchain_community.document_loaders import CSVLoader, UnstructuredCSVLoader


#METHOD 1: EACH ROW AS A DOCUMENT
csv_loader = loaders.CSVLoader(file_path='data/structured_files/sample_data.csv', csv_args={'delimiter': ',', 'quotechar': '"'})
csv_documents = csv_loader.load()
print(f"Loaded {len(csv_documents)} documents from CSV file ONE PER ROW .")
for i, doc in enumerate(csv_documents[:3]):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")





Loaded 3 documents from CSV file ONE PER ROW .
Document 1:
Content: Name: Alice
Age: 25
City: New York
Metadata: {'source': 'data/structured_files/sample_data.csv', 'row': 0}
Document 2:
Content: Name: Bob
Age: 30
City: Los Angeles
Metadata: {'source': 'data/structured_files/sample_data.csv', 'row': 1}
Document 3:
Content: Name: Charlie
Age: 35
City: Chicago
Metadata: {'source': 'data/structured_files/sample_data.csv', 'row': 2}
Loaded 3 documents from Excel file ONE PER COLUMN.
Document 1:
Content: Name: Alice
Age: 25
City: New York
Metadata: {'source': 'data/structured_files/sample_data.CSV  ', 'row': 0}
Document 2:
Content: Name: Bob
Age: 30
City: Los Angeles
Metadata: {'source': 'data/structured_files/sample_data.CSV  ', 'row': 1}
Document 3:
Content: Name: Charlie
Age: 35
City: Chicago
Metadata: {'source': 'data/structured_files/sample_data.CSV  ', 'row': 2}


In [ ]:
#METHOD 2: UnstructuredCSVLoader
unstructured_csv_loader = UnstructuredCSVLoader(file_path='data/structured_files/sample_data.csv', csv_args={'delimiter': ',', 'quotechar': '"'})
unstructured_csv_documents = unstructured_csv_loader.load()
print(f"Loaded {len(unstructured_csv_documents)} documents from CSV file using UnstructuredCSV
Loader.")
for i, doc in enumerate(unstructured_csv_documents[:3]):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")


In [32]:
#custom CSV loader processing for better handling of structured data
from typing import List
from langchain_core.documents import Document
import pandas as pd
class CustomCSVLoader:
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        df = pd.read_csv(self.file_path)
        documents = []
        #Strategy 1 One document per row with Strutured content
        # Iterate through each row in the DataFrame 
        for index, row in df.iterrows():
            content = ' '.join([str(value) for value in row.values])
            metadata = {col: row[col] for col in df.columns}
            documents.append(Document(page_content=content, metadata=metadata))
        return documents
# Load CSV file using custom loader
loader = CustomCSVLoader(file_path='data/structured_files/sample_data.csv')
documents = loader.load()
print(f"Loaded {len(documents)} documents using CustomCSVLoader.")
for i, doc in enumerate(documents[:3]):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")

Loaded 3 documents using CustomCSVLoader.
Document 1:
Content: Alice 25 New York
Metadata: {'Name': 'Alice', 'Age': 25, 'City': 'New York'}
Document 2:
Content: Bob 30 Los Angeles
Metadata: {'Name': 'Bob', 'Age': 30, 'City': 'Los Angeles'}
Document 3:
Content: Charlie 35 Chicago
Metadata: {'Name': 'Charlie', 'Age': 35, 'City': 'Chicago'}


### Excel Procssing 

In [36]:
#Method 1 using pandas for ful control
import pandas as pd
# Load xlsx file using pandas
class CustomExcelLoader:
    def __init__(self, file_path, sheet_name='Sheet1'):
        self.file_path = file_path
        self.sheet_name = sheet_name

    def load(self) -> List[Document]:
        df = pd.read_excel(self.file_path, sheet_name=self.sheet_name)
        documents = []
        # Iterate through each row in the DataFrame
        for index, row in df.iterrows():
            content = ' '.join([str(value) for value in row.values])
            metadata = {col: row[col] for col in df.columns}
            documents.append(Document(page_content=content, metadata=metadata))
        return documents

# Load Excel file using custom loader
excel_loader = CustomExcelLoader(file_path='data/structured_files/sample_data.xlsx', sheet_name='Sheet1')
excel_documents = excel_loader.load()
print(f"Loaded {len(excel_documents)} documents using CustomExcelLoader.")
for i, doc in enumerate(excel_documents[:3]):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")

#loading each sheet as a separate document
class CustomExcelMultiSheetLoader:
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self) -> List[Document]:
        xls = pd.ExcelFile(self.file_path)
        documents = []
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            for index, row in df.iterrows():
                content = ' '.join([str(value) for value in row.values])
                metadata = {col: row[col] for col in df.columns}
                metadata['sheet_name'] = sheet_name
                documents.append(Document(page_content=content, metadata=metadata))
        return documents
# Load Excel file with multiple sheets using custom loader
multi_sheet_loader = CustomExcelMultiSheetLoader(file_path='data/structured_files/sample_data_multiple_sheets.xlsx')
multi_sheet_documents = multi_sheet_loader.load()
print(f"Loaded {len(multi_sheet_documents)} documents from Excel file with multiple sheets using CustomExcelMultiSheetLoader.")
for i, doc in enumerate(multi_sheet_documents[:3]):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print(f"Sheet Name: {doc.metadata.get('sheet_name', 'N/A')}")
    

Loaded 3 documents using CustomExcelLoader.
Document 1:
Content: Alice 25 New York
Metadata: {'Name': 'Alice', 'Age': 25, 'City': 'New York'}
Document 2:
Content: Bob 30 Los Angeles
Metadata: {'Name': 'Bob', 'Age': 30, 'City': 'Los Angeles'}
Document 3:
Content: Charlie 35 Chicago
Metadata: {'Name': 'Charlie', 'Age': 35, 'City': 'Chicago'}
Loaded 6 documents from Excel file with multiple sheets using CustomExcelMultiSheetLoader.
Document 1:
Content: Alice 25 New York
Metadata: {'Name': 'Alice', 'Age': 25, 'City': 'New York', 'sheet_name': 'Sheet1'}
Sheet Name: Sheet1
Document 2:
Content: Bob 30 Los Angeles
Metadata: {'Name': 'Bob', 'Age': 30, 'City': 'Los Angeles', 'sheet_name': 'Sheet1'}
Sheet Name: Sheet1
Document 3:
Content: Charlie 35 Chicago
Metadata: {'Name': 'Charlie', 'Age': 35, 'City': 'Chicago', 'sheet_name': 'Sheet1'}
Sheet Name: Sheet1


### Unstructure excel loader

In [46]:
# Unstructured Excel Loader
from langchain_community.document_loaders import UnstructuredExcelLoader
try:
    # Load the Excel file using UnstructuredExcelLoader
    unstructured_excel_loader = UnstructuredExcelLoader(
        file_path="data/structured_files/sample_data_multiple_sheets.xlsx"
    )
    unstructured_excel_documents = unstructured_excel_loader.load()

    print(f"Loaded {len(unstructured_excel_documents)} documents from Excel file using UnstructuredExcelLoader.\n")

    # Show first 3 docs for inspection
    for i, doc in enumerate(unstructured_excel_documents[:3], start=1):
        print(f"--- Document {i} ---")
        print(f"Content:\n{doc.page_content[:500]}...")  # only preview first 500 chars
        print(f"Metadata: {doc.metadata}\n")
except Exception as e:
    print(f"Error loading documents with UnstructuredExcelLoader: {e}")
    



Loaded 1 documents from Excel file using UnstructuredExcelLoader.

--- Document 1 ---
Content:
Name Age City Alice 25 New York Bob 30 Los Angeles Charlie 35 Chicago

Name Age City Alice 25 New York Bob 30 Los Angeles Charlie 35 Chicago...
Metadata: {'source': 'data/structured_files/sample_data_multiple_sheets.xlsx'}

